# Artificial Intelligence– A1(IITJ)
**Prepared by :**
**Mahantesh Hiremath- G24AIT2178**

A.) Dynamic Goal-Based Agent for Warehouse Logistics Optimization.
A robotic agent operates in a warehouse modeled as an N×M grid environment. The agent starts at a predefined loading dock and must deliver packages to multiple destinations marked on the grid while avoiding dynamically placed obstacles.
Take suitable values of the following parameters.
- Warehouse dimensions: N×M grid size (M,N between 5 and 10, inclusive)
- Number of packages: P (between 2 and 6, inclusive)
- Number of obstacles: O (between 1 and 10, inclusive)
- Package locations: (X1, Y1), (X2, Y2), ... (XP, YP)
- Drop-off locations: (D1X, D1Y), (D2X, D2Y), ... (DPX, DPY)
- Robot starting position: S=(Sx,Sy), starts at a fixed cell but moves dynamically
- Movement cost: Each movement incurs a cost of 1 unit
- Delivery reward: Successfully delivering a package adds 10 units to the total reward
- Obstacle penalty: Hitting an obstacle results in a (-5) penalty
Note : Packages locations and drop-off locations should not overlap.




# Q1. Represent the warehouse as an N×M matrix. Place the packages, drop-off points, and obstacles randomly. Display the initial warehouse configuration.


In [2]:
!pip install numpy

In [4]:
import numpy as np

# Parameters
N, M = 8, 8  # Warehouse dimensions
P = 4  # Number of packages
O = 5  # Number of obstacles

# Initialize the warehouse grid
warehouse = np.full((N, M), '.')

# Randomly place packages and drop-off points
package_locations = []
dropoff_locations = []

for i in range(P):
    while True:
        # Randomly select package location
        package = (np.random.randint(0, N), np.random.randint(0, M))
        # Randomly select drop-off location
        dropoff = (np.random.randint(0, N), np.random.randint(0, M))
        
        # Ensure package and drop-off locations do not overlap
        if package != dropoff and package not in package_locations and dropoff not in dropoff_locations:
            package_locations.append(package)
            dropoff_locations.append(dropoff)
            break

# Place packages and drop-off points on the grid
for idx, (x, y) in enumerate(package_locations):
    warehouse[x][y] = f'P{idx+1}'  # Mark packages as P1, P2, etc.

for idx, (x, y) in enumerate(dropoff_locations):
    warehouse[x][y] = f'D{idx+1}'  # Mark drop-off points as D1, D2, etc.

# Randomly place obstacles
obstacle_locations = []
for _ in range(O):
    while True:
        obstacle = (np.random.randint(0, N), np.random.randint(0, M))
        # Ensure obstacles do not overlap with packages, drop-off points, or other obstacles
        if warehouse[obstacle[0]][obstacle[1]] == '.':
            warehouse[obstacle[0]][obstacle[1]] = 'O'  # Mark obstacles as 'O'
            obstacle_locations.append(obstacle)
            break

# Display the initial warehouse configuration
print("Initial Warehouse Configuration:")
print(warehouse)

# Print locations for reference
print("\nPackage Locations:", package_locations)
print("Drop-off Locations:", dropoff_locations)
print("Obstacle Locations:", obstacle_locations)

Initial Warehouse Configuration:
[['.' '.' '.' '.' '.' '.' 'O' '.']
 ['.' '.' '.' '.' '.' '.' 'D' 'O']
 ['.' '.' 'P' '.' 'P' 'D' '.' '.']
 ['.' '.' '.' '.' '.' 'O' '.' 'D']
 ['.' 'D' '.' '.' '.' '.' '.' '.']
 ['O' '.' '.' '.' '.' '.' 'O' 'P']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]

Package Locations: [(3, 7), (5, 7), (2, 4), (2, 2)]
Drop-off Locations: [(4, 1), (2, 5), (1, 6), (3, 7)]
Obstacle Locations: [(0, 6), (1, 7), (5, 6), (3, 5), (5, 0)]


# Q2. Implement a goal-based agent that can identify all goals, plan a sequence of actions to reach the goal, use a search algorithm (BFS, DFS, or UCS) to find optimal paths, deliver all packages, and calculate the total cost.

In [5]:
import numpy as np
from queue import PriorityQueue

# Warehouse parameters
N, M = 8, 8  # Warehouse dimensions
P = 4  # Number of packages
O = 5  # Number of obstacles

# Initialize the warehouse grid
warehouse = np.full((N, M), '.')

# Randomly place packages and drop-off points
package_locations = []
dropoff_locations = []

for i in range(P):
    while True:
        # Randomly select package location
        package = (np.random.randint(0, N), np.random.randint(0, M))
        # Randomly select drop-off location
        dropoff = (np.random.randint(0, N), np.random.randint(0, M))
        
        # Ensure package and drop-off locations do not overlap
        if package != dropoff and package not in package_locations and dropoff not in dropoff_locations:
            package_locations.append(package)
            dropoff_locations.append(dropoff)
            break

# Place packages and drop-off points on the grid
for idx, (x, y) in enumerate(package_locations):
    warehouse[x][y] = f'P{idx+1}'  # Mark packages as P1, P2, etc.

for idx, (x, y) in enumerate(dropoff_locations):
    warehouse[x][y] = f'D{idx+1}'  # Mark drop-off points as D1, D2, etc.

# Randomly place obstacles
obstacle_locations = []
for _ in range(O):
    while True:
        obstacle = (np.random.randint(0, N), np.random.randint(0, M))
        # Ensure obstacles do not overlap with packages, drop-off points, or other obstacles
        if warehouse[obstacle[0]][obstacle[1]] == '.':
            warehouse[obstacle[0]][obstacle[1]] = 'O'  # Mark obstacles as 'O'
            obstacle_locations.append(obstacle)
            break

# Display the initial warehouse configuration
print("Initial Warehouse Configuration:")
print(warehouse)

# Print locations for reference
print("\nPackage Locations:", package_locations)
print("Drop-off Locations:", dropoff_locations)
print("Obstacle Locations:", obstacle_locations)

# Define the agent's starting position
start_position = (0, 0)  # Loading dock

# Movement directions (up, down, left, right)
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Uniform Cost Search (UCS) algorithm
def ucs(start, goal, grid):
    frontier = PriorityQueue()
    frontier.put((0, start))  # (cost, position)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current_cost, current_pos = frontier.get()

        if current_pos == goal:
            break

        for direction in directions:
            next_pos = (current_pos[0] + direction[0], current_pos[1] + direction[1])
            if 0 <= next_pos[0] < N and 0 <= next_pos[1] < M:  # Check boundaries
                if grid[next_pos[0]][next_pos[1]] == 'O':  # Obstacle
                    continue
                new_cost = current_cost + 1  # Movement cost is 1
                if next_pos not in cost_so_far or new_cost < cost_so_far[next_pos]:
                    cost_so_far[next_pos] = new_cost
                    priority = new_cost
                    frontier.put((priority, next_pos))
                    came_from[next_pos] = current_pos

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path, cost_so_far[goal]

# Agent logic
total_cost = 0
total_reward = 0
current_position = start_position

for i in range(P):
    package = package_locations[i]
    dropoff = dropoff_locations[i]

    # Plan path from current position to package
    path_to_package, cost_to_package = ucs(current_position, package, warehouse)
    print(f"\nPath to Package {i+1}: {path_to_package}")
    print(f"Cost to Package {i+1}: {cost_to_package}")
    total_cost += cost_to_package

    # Plan path from package to drop-off
    path_to_dropoff, cost_to_dropoff = ucs(package, dropoff, warehouse)
    print(f"Path to Drop-off {i+1}: {path_to_dropoff}")
    print(f"Cost to Drop-off {i+1}: {cost_to_dropoff}")
    total_cost += cost_to_dropoff

    # Update current position
    current_position = dropoff

    # Add delivery reward
    total_reward += 10

# Calculate total reward after penalties
total_penalty = 0  # Assuming no obstacles are hit in the path
final_reward = total_reward - total_cost - total_penalty

print("\nTotal Cost:", total_cost)
print("Total Reward:", total_reward)
print("Final Reward (Reward - Cost - Penalty):", final_reward)

Initial Warehouse Configuration:
[['.' '.' 'D' '.' '.' '.' '.' '.']
 ['.' '.' '.' 'P' '.' '.' '.' '.']
 ['.' '.' 'O' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' 'O' '.' '.' '.']
 ['.' 'P' '.' '.' '.' '.' 'P' 'D']
 ['O' 'O' '.' '.' '.' '.' '.' 'D']
 ['.' '.' '.' '.' '.' 'D' '.' '.']
 ['.' '.' 'O' '.' '.' '.' '.' 'P']]

Package Locations: [(1, 3), (4, 1), (7, 7), (4, 6)]
Drop-off Locations: [(6, 5), (5, 7), (4, 7), (0, 2)]
Obstacle Locations: [(3, 4), (5, 1), (7, 2), (2, 2), (5, 0)]

Path to Package 1: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3)]
Cost to Package 1: 4
Path to Drop-off 1: [(1, 3), (1, 4), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5)]
Cost to Drop-off 1: 7

Path to Package 2: [(6, 5), (5, 5), (4, 5), (4, 4), (4, 3), (4, 2), (4, 1)]
Cost to Package 2: 6
Path to Drop-off 2: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 7)]
Cost to Drop-off 2: 7

Path to Package 3: [(5, 7), (6, 7), (7, 7)]
Cost to Package 3: 2
Path to Drop-off 3: [(7, 7), (6, 7), (5, 7), (4, 7)]
Cost t

# Q3. Choose a random seed value for the ease of reproducing the results. Your program should give outputs: the chosen path taken by the agent, total cost and rewards, final score based on penalties, movement costs, and successful deliveries.

In [7]:
import numpy as np
from queue import PriorityQueue

# Set a random seed for reproducibility
np.random.seed(42)

# Warehouse parameters
N, M = 8, 8  # Warehouse dimensions
P = 4  # Number of packages
O = 5  # Number of obstacles

# Initialize the warehouse grid
warehouse = np.full((N, M), '.')

# Randomly place packages and drop-off points
package_locations = []
dropoff_locations = []

for i in range(P):
    while True:
        # Randomly select package location
        package = (np.random.randint(0, N), np.random.randint(0, M))
        # Randomly select drop-off location
        dropoff = (np.random.randint(0, N), np.random.randint(0, M))
        
        # Ensure package and drop-off locations do not overlap
        if package != dropoff and package not in package_locations and dropoff not in dropoff_locations:
            package_locations.append(package)
            dropoff_locations.append(dropoff)
            break

# Place packages and drop-off points on the grid
for idx, (x, y) in enumerate(package_locations):
    warehouse[x][y] = f'P{idx+1}'  # Mark packages as P1, P2, etc.

for idx, (x, y) in enumerate(dropoff_locations):
    warehouse[x][y] = f'D{idx+1}'  # Mark drop-off points as D1, D2, etc.

# Randomly place obstacles
obstacle_locations = []
for _ in range(O):
    while True:
        obstacle = (np.random.randint(0, N), np.random.randint(0, M))
        # Ensure obstacles do not overlap with packages, drop-off points, or other obstacles
        if warehouse[obstacle[0]][obstacle[1]] == '.':
            warehouse[obstacle[0]][obstacle[1]] = 'O'  # Mark obstacles as 'O'
            obstacle_locations.append(obstacle)
            break

# Display the initial warehouse configuration
print("Initial Warehouse Configuration:")
print(warehouse)

# Print locations for reference
print("\nPackage Locations:", package_locations)
print("Drop-off Locations:", dropoff_locations)
print("Obstacle Locations:", obstacle_locations)

# Define the agent's starting position
start_position = (0, 0)  # Loading dock

# Movement directions (up, down, left, right)
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Uniform Cost Search (UCS) algorithm
def ucs(start, goal, grid):
    frontier = PriorityQueue()
    frontier.put((0, start))  # (cost, position)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current_cost, current_pos = frontier.get()

        if current_pos == goal:
            break

        for direction in directions:
            next_pos = (current_pos[0] + direction[0], current_pos[1] + direction[1])
            if 0 <= next_pos[0] < N and 0 <= next_pos[1] < M:  # Check boundaries
                if grid[next_pos[0]][next_pos[1]] == 'O':  # Obstacle
                    continue
                new_cost = current_cost + 1  # Movement cost is 1
                if next_pos not in cost_so_far or new_cost < cost_so_far[next_pos]:
                    cost_so_far[next_pos] = new_cost
                    priority = new_cost
                    frontier.put((priority, next_pos))
                    came_from[next_pos] = current_pos

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path, cost_so_far[goal]

# Agent logic
total_cost = 0
total_reward = 0
total_penalty = 0
current_position = start_position

for i in range(P):
    package = package_locations[i]
    dropoff = dropoff_locations[i]

    # Plan path from current position to package
    path_to_package, cost_to_package = ucs(current_position, package, warehouse)
    print(f"\nPath to Package {i+1}: {path_to_package}")
    print(f"Cost to Package {i+1}: {cost_to_package}")
    total_cost += cost_to_package

    # Plan path from package to drop-off
    path_to_dropoff, cost_to_dropoff = ucs(package, dropoff, warehouse)
    print(f"Path to Drop-off {i+1}: {path_to_dropoff}")
    print(f"Cost to Drop-off {i+1}: {cost_to_dropoff}")
    total_cost += cost_to_dropoff

    # Update current position
    current_position = dropoff

    # Add delivery reward
    total_reward += 10

# Calculate final score
final_score = total_reward - total_cost - total_penalty

# Output results
print("\n--- Final Results ---")
print(f"Total Cost: {total_cost}")
print(f"Total Reward: {total_reward}")
print(f"Total Penalty: {total_penalty}")
print(f"Final Score (Reward - Cost - Penalty): {final_score}")

Initial Warehouse Configuration:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' 'O']
 ['.' '.' 'P' '.' '.' '.' 'D' 'P']
 ['.' '.' '.' '.' '.' 'O' '.' 'O']
 ['.' '.' '.' '.' 'D' '.' 'D' '.']
 ['.' '.' '.' '.' 'O' '.' '.' '.']
 ['.' 'P' '.' 'P' '.' '.' '.' '.']
 ['.' '.' 'O' '.' 'D' '.' '.' '.']]

Package Locations: [(6, 3), (2, 7), (6, 1), (2, 2)]
Drop-off Locations: [(4, 6), (4, 4), (2, 6), (7, 4)]
Obstacle Locations: [(3, 7), (7, 2), (5, 4), (1, 7), (3, 5)]

Path to Package 1: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3)]
Cost to Package 1: 9
Path to Drop-off 1: [(6, 3), (5, 3), (4, 3), (4, 4), (4, 5), (4, 6)]
Cost to Drop-off 1: 5

Path to Package 2: [(4, 6), (3, 6), (2, 6), (2, 7)]
Cost to Package 2: 3
Path to Drop-off 2: [(2, 7), (2, 6), (2, 5), (2, 4), (3, 4), (4, 4)]
Cost to Drop-off 2: 5

Path to Package 3: [(4, 4), (4, 3), (4, 2), (4, 1), (5, 1), (6, 1)]
Cost to Package 3: 5
Path to Drop-off 3: [(6, 1), (5, 1), (4, 1), (3, 1)